In [2]:
import time
import os
import win32com.client

class mySAP:
    def __init__(self, utente=None, password=None):
        self.apri_sap()
        self.session = self.connetti_sap()
        if utente:
            self.logon(utente, password)
        
    def apri_sap(self):
        # Avvia SAP GUI
        os.system("start saplogon.exe")
        time.sleep(5)  # Aspetta che SAP GUI si avvii
        
    def connetti_sap(self):
        # Connetti a SAP GUI tramite COM
        sapguiapp = win32com.client.GetObject("SAPGUI").GetScriptingEngine
        if sapguiapp.Children.Count > 0:
            connection = sapguiapp.Children(0)
        else:
            connection = sapguiapp.OpenConnection('INE [PUBLIC]')
        session = connection.Children(0)
        return session

    def logon(self, utente, password):
        self.session.findbyid('wnd[0]/usr/txtRSYST-BNAME').text = utente
        self.session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = password
        
mysap = mySAP('932197', '.Gioia7777')
mysap.session.StartTransaction('CN22')


'  /SAP_CODEPAGE=1100  /FULLMENU  /M/ine00.rfi.it/S/sapmsINE/G/PUBLIC /UPDOWNLOAD_CP=2'